<a href="https://colab.research.google.com/github/NataliaGon/kpi/blob/AI-cybersecurity/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import os
import pandas as pd
import kagglehub
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

In [33]:
path = kagglehub.dataset_download("whenamancodes/infoseccyber-security-salaries")

print("Path to dataset files:", path)
print("Files in dataset folder:", os.listdir(path))

Using Colab cache for faster access to the 'infoseccyber-security-salaries' dataset.
Path to dataset files: /kaggle/input/infoseccyber-security-salaries
Files in dataset folder: ['Cyber_salaries.csv']


In [34]:
df = pd.read_csv(os.path.join(path, "Cyber_salaries.csv"))

print("Size rows/columns:", df.shape)

print(df.isnull().sum())

df.describe()


Size rows/columns: (1349, 11)
work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64


,work_year,salary,salary_in_usd,remote_ratio
count,1349.000000,1.349000e+03,1349.000000,1349.000000
mean,2021.368421,5.288248e+05,121578.622683,72.238695
std,0.711236,1.361367e+07,68972.954838,39.355331
min,2020.000000,1.740000e+03,2000.000000,0.000000
25%,2021.000000,8.000000e+04,75000.000000,50.000000
50%,2022.000000,1.200000e+05,115000.000000,100.000000
75%,2022.000000,1.600800e+05,152000.000000,100.000000
max,2022.000000,5.000000e+08,899920.000000,100.000000


In [25]:
target = "experience_level"
drop_cols = [
    "salary","salary_currency"
]


X = df.drop(columns=drop_cols + [target])
y = df[target]


for col in X.select_dtypes(include=["object"]).columns:
    X[col] = LabelEncoder().fit_transform(X[col])


scaler = StandardScaler()
X[["salary_in_usd", "remote_ratio"]] = scaler.fit_transform(X[["salary_in_usd", "remote_ratio"]])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
random_state=42)

print("Train size:", X_train.shape)
print("Test size:", X.describe())

Train size: (1079, 8)
Test size:          work_year  employment_type    job_title  salary_in_usd  \
count  1349.000000      1349.000000  1349.000000    1349.000000   
mean   2021.368421         1.988139    43.484062       0.000000   
std       0.711236         0.203475    23.353002       1.000371   
min    2020.000000         0.000000     0.000000      -1.734346   
25%    2021.000000         2.000000    19.000000      -0.675568   
50%    2022.000000         2.000000    46.000000      -0.095415   
75%    2022.000000         2.000000    68.000000       0.441226   
max    2022.000000         3.000000    86.000000      11.288918   

       employee_residence  remote_ratio  company_location  company_size  
count         1349.000000  1.349000e+03       1349.000000   1349.000000  
mean            46.836916  1.158780e-16         45.170497      0.476649  
std             17.231883  1.000371e+00         15.504206      0.621400  
min              0.000000 -1.836231e+00          0.000000      0.00

In [28]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)


y_pred = knn.predict(X_test)

In [29]:
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy', accuracy)

Accuracy 0.5259259259259259


In [31]:
knn = KNeighborsClassifier()

param_grid = {
    "n_neighbors": [3,5,7,9,11,15,21],
    "weights": ["uniform", "distance"],
    "metric": ["minkowski"],   # with p 1 (Manhattan) or 2 (Euclidean)
    "p": [1, 2],
    "leaf_size": [15, 30, 50]
}

grid = GridSearchCV(
    estimator=knn,
    param_grid=param_grid,
    cv=5,
    scoring="accuracy",       # try "balanced_accuracy" if classes imbalanced
    n_jobs=-1,
    verbose=1
)

grid.fit(X_train, y_train)
print("Best params:", grid.best_params_)
print("Best CV accuracy:", grid.best_score_)

best_knn = grid.best_estimator_
y_pred = best_knn.predict(X_test)

print("Test accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification report:\n", classification_report(y_test, y_pred))
print("\nConfusion matrix:\n", confusion_matrix(y_test, y_pred))


Fitting 5 folds for each of 84 candidates, totalling 420 fits
Best params: {'leaf_size': 15, 'metric': 'minkowski', 'n_neighbors': 21, 'p': 1, 'weights': 'distance'}
Best CV accuracy: 0.5727260981912146
Test accuracy: 0.5925925925925926

Classification report:
               precision    recall  f1-score   support

          EN       0.48      0.32      0.38        41
          EX       0.71      0.26      0.38        19
          MI       0.60      0.52      0.56        92
          SE       0.60      0.80      0.69       118

    accuracy                           0.59       270
   macro avg       0.60      0.47      0.50       270
weighted avg       0.59      0.59      0.58       270


Confusion matrix:
 [[13  0 11 17]
 [ 0  5  3 11]
 [10  0 48 34]
 [ 4  2 18 94]]


# Висновки

У межах даної лабораторної роботи було проведено налаштування гіперпараметрів методу k-найближчих сусідів (k-Nearest Neighbors, k-NN) для класифікації рівня досвіду фахівців у галузі кібербезпеки на основі даних із відкритого набору Cyber Security Salaries (Kaggle).

Для попередньої обробки даних було виконано нормалізацію числових ознак та кодування категоріальних змінних. Далі застосовано GridSearchCV для пошуку оптимальних параметрів моделі. Найкращі результати було отримано при таких гіперпараметрах:

n_neighbors = 21, weights = "distance", metric = "minkowski", p = 1, leaf_size = 15.
Після навчання на тренувальній вибірці (80 % даних) і тестування на 20 % залишку даних модель показала:
Середню точність (accuracy) на тестовій вибірці — 0.593 (≈59 %);

Найкраще розпізнавання для класу Senior (точність = 0.60, повнота = 0.80, F1 = 0.69);

Найгірші результати для класів Entry та Expert, що свідчить про їхню часту плутанину з іншими категоріями.

Аналіз матриці плутанини показав, що більшість помилок моделі пов’язана з перекриттям характеристик між сусідніми рівнями досвіду — зокрема, «Entry» та «Mid» або «Senior» та «Expert». Це зумовлено тим, що доступні ознаки (зарплата, віддаленість, місце роботи тощо) не відображають повною мірою реальний професійний рівень працівника.

Отримані результати свідчать, що метод k-найближчих сусідів забезпечує лише середню якість класифікації для даного набору, що пояснюється наявністю великої кількості категоріальних змінних, їх високою розмірністю після one-hot кодування та можливою дисбалансованістю класів.